# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_df = pd.read_csv("city_weather.csv")

weather_df.head()

,city name,cloudiness,country,date,max_temp,humidity,lat,lng,wind_speed
0,Mataura,12,NZ,1644254715,43.48,99,-46.1927,168.8643,1.45
1,Kamennogorsk,100,RU,1644254716,32.81,97,60.9545,29.1339,6.49
2,Pevek,23,RU,1644254716,-27.02,98,69.7008,170.3133,2.24
3,Birin,96,NP,1644254717,44.24,78,26.8865,88.0718,0.74
4,Lompoc,0,US,1644254467,75.13,50,34.6391,-120.4579,3.44


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

location = weather_df[["lat", "lng"]]

humidity = weather_df["humidity"].astype(float)

In [35]:
fig = gmaps.figure()
fig = gmaps.figure(center=(20.0, 10), zoom_level=2)
# Create heat layer
heat_layer = gmaps.heatmap_layer(location, weights=humidity, 
                                 dissipating=False, max_intensity=300,
                                 point_radius=5)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
weather_df=weather_df.dropna()
weather_df.head()

,city name,cloudiness,country,date,max_temp,humidity,lat,lng,wind_speed
0,Mataura,12,NZ,1644254715,43.48,99,-46.1927,168.8643,1.45
1,Kamennogorsk,100,RU,1644254716,32.81,97,60.9545,29.1339,6.49
2,Pevek,23,RU,1644254716,-27.02,98,69.7008,170.3133,2.24
3,Birin,96,NP,1644254717,44.24,78,26.8865,88.0718,0.74
4,Lompoc,0,US,1644254467,75.13,50,34.6391,-120.4579,3.44


In [6]:
df = weather_df[(weather_df['max_temp'] >= 70) & (weather_df['max_temp'] <=85) & 
                (weather_df['wind_speed'] <= 12) & (weather_df['cloudiness'] == 0)]
df

,city name,cloudiness,country,date,max_temp,humidity,lat,lng,wind_speed
4,Lompoc,0,US,1644254467,75.13,50,34.6391,-120.4579,3.44
24,Coyhaique,0,CL,1644254723,77.61,31,-45.5752,-72.0662,11.50
70,Port Elizabeth,0,ZA,1644254738,72.10,78,-33.9180,25.5701,5.75
178,Marsh Harbour,0,BS,1644254771,75.09,67,26.5412,-77.0636,11.52
229,Makakilo City,0,US,1644254658,71.92,69,21.3469,-158.0858,10.36
362,Cabo San Lucas,0,MX,1644254782,74.61,54,22.8909,-109.9124,0.00
427,San Isidro,0,PE,1644254833,81.01,58,-12.1167,-77.0500,8.05
437,Luanda,0,AO,1644254854,82.40,61,-8.8368,13.2343,10.36
501,Salalah,0,OM,1644254724,78.89,17,17.0151,54.0924,11.50
511,Puerto Escondido,0,MX,1644254878,83.23,74,15.8500,-97.0667,4.61


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df=df

In [8]:
hotel_names = []
lat = df['lat']
lng = df['lng']
lat_lngs = zip(lat,lng)

for lat_lng in lat_lngs:

    target_coordinates = f"{lat_lng[0]},{lat_lng[1]}"
    target_radius = 5000
    target_type = "hotel"


    params = {
        "location": target_coordinates,
        "radius": target_radius,
        "types": target_type,
        "key": g_key
    }

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    response = requests.get(base_url, params=params)
      
    places_data = response.json()

    try:
        hotel_names.append(places_data["results"][1]["name"])
    except:
        hotel_names.append("")
        
hotel_names
hotel_df["Hotel Name"] = hotel_names


C:\Users\manni\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [9]:
hotel_df = hotel_df.rename(columns={"city name": "City", "lat": "Lat", "lng": "Lng", "country": "Country"})
hotel_df

,City,cloudiness,Country,date,max_temp,humidity,Lat,Lng,wind_speed,Hotel Name
4,Lompoc,0,US,1644254467,75.13,50,34.6391,-120.4579,3.44,"Holiday Inn Express Lompoc, an IHG Hotel"
24,Coyhaique,0,CL,1644254723,77.61,31,-45.5752,-72.0662,11.50,Diego De Almagro Coyhaique
70,Port Elizabeth,0,ZA,1644254738,72.10,78,-33.9180,25.5701,5.75,Gelvandale Stadium
178,Marsh Harbour,0,BS,1644254771,75.09,67,26.5412,-77.0636,11.52,Abaco Beach Resort
229,Makakilo City,0,US,1644254658,71.92,69,21.3469,-158.0858,10.36,Wet 'n' Wild Hawaii
362,Cabo San Lucas,0,MX,1644254782,74.61,54,22.8909,-109.9124,0.00,Hotel Tesoro Los Cabos
427,San Isidro,0,PE,1644254833,81.01,58,-12.1167,-77.0500,8.05,Soul Mate-Inn
437,Luanda,0,AO,1644254854,82.40,61,-8.8368,13.2343,10.36,EPIC SANA Luanda
501,Salalah,0,OM,1644254724,78.89,17,17.0151,54.0924,11.50,"HAMDAN PLAZA HOTEL SALALAH, an HTG Hotel"
511,Puerto Escondido,0,MX,1644254878,83.23,74,15.8500,-97.0667,4.61,Aldea del Bazar Hotel and Spa


In [30]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [31]:
# Add marker layer ontop of heat map
fig = gmaps.figure()
fig = gmaps.figure(center=(0, 10), zoom_level=2)
#
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
#infowindow.open(map,marker)
fig.add_layer(markers)
# Add layer
fig.add_layer(heat_layer)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))